In [6]:
# 원화마켓o, 유의종목x, BTCx 코인에 대해서
# 실시간으로 1분봉의 정보들을 가져오고
# 1초마다 체크해서
# 1분안에 이전 1분봉대비 10%가 오른 코인에 대해서
# 원화의 20%를 시장가매수하고
# 1초마다 체크해서
# 5%이상의 수익이 나오면 시장가매도를 한다.
# 계속 반복한다.

In [7]:
import requests, time, jwt, uuid, pyupbit, datetime
import pandas as pd
import numpy as np
from sendMail import send_email
import json

f = open("업비트정보.private.json", "r")
api_key = json.load(f)
f.close()

A_key = api_key["accessKey"]  # 본인 access_key 키로 변경
S_key = api_key["secretKey"]  # 본인 secret_key 키로 변경

In [12]:
def get_high_volume_tickers():
    result = []
    for t in pyupbit.get_tickers(fiat='KRW'):
        volumes = (t, pyupbit.get_ohlcv(ticker=t, count=1)['value'].values[0] )
        result.append(volumes)
        time.sleep(0.1)

    result.sort(key=lambda x:x[1])
        
    return list(map(lambda x: x[0], result[:-20]))

tickers = get_high_volume_tickers()
        
## 몇 프로 이상 상승하면 수익화 할 것인지
ascent = 5  # 5% 상승
## 몇 프로 이상 하락하면 손절 할 것인지
downhill = 5  # 5% 하락

In [13]:
tickers

['KRW-BTT',
 'KRW-AAVE',
 'KRW-XLM',
 'KRW-HBAR',
 'KRW-ELF',
 'KRW-THETA',
 'KRW-BCH',
 'KRW-ZRX',
 'KRW-STPT',
 'KRW-XEM',
 'KRW-RFR',
 'KRW-NU',
 'KRW-VET',
 'KRW-KAVA',
 'KRW-LINK',
 'KRW-IOST',
 'KRW-DOT',
 'KRW-TT',
 'KRW-MVL',
 'KRW-AVAX',
 'KRW-CRO',
 'KRW-BAT',
 'KRW-IOTA',
 'KRW-DKA',
 'KRW-ALGO',
 'KRW-JST',
 'KRW-XEC',
 'KRW-TRX',
 'KRW-MBL',
 'KRW-AHT',
 'KRW-CHZ',
 'KRW-NEO',
 'KRW-HIFI',
 'KRW-ANKR',
 'KRW-EOS',
 'KRW-QKC',
 'KRW-STMX',
 'KRW-ONG',
 'KRW-CVC',
 'KRW-ZIL',
 'KRW-AERGO',
 'KRW-SNT',
 'KRW-ATOM',
 'KRW-XTZ',
 'KRW-BTG',
 'KRW-ONT',
 'KRW-1INCH',
 'KRW-SC',
 'KRW-FCT2',
 'KRW-META',
 'KRW-ADA',
 'KRW-MATIC',
 'KRW-STEEM',
 'KRW-SXP',
 'KRW-NEAR',
 'KRW-LOOM',
 'KRW-STX',
 'KRW-QTUM',
 'KRW-TFUEL',
 'KRW-ETH',
 'KRW-SOL',
 'KRW-T',
 'KRW-MLK',
 'KRW-IQ',
 'KRW-SSX',
 'KRW-ORBS',
 'KRW-ICX',
 'KRW-SRM',
 'KRW-POLYX',
 'KRW-BSV',
 'KRW-GMT',
 'KRW-BORA',
 'KRW-GLM',
 'KRW-ENJ',
 'KRW-MOC',
 'KRW-SBD',
 'KRW-MANA',
 'KRW-FLOW',
 'KRW-STRAX',
 'KRW-PLA',
 'KRW-HI

In [ ]:
while(1):
    for market_code in tickers:
        time.sleep(0.1)

        ## BTT는 너무 단위가 작아서..
        if market_code in ["KRW-BTC", "KRW-ETH", "KRW-XRP", "KRW-BTT"]:
            continue

        print(market_code, f'{pyupbit.get_current_price(market_code)}원')

        ## API로 업비트에서 내 계좌 조회
        my_exchange_account = pd.DataFrame(requests.get("https://api.upbit.com/v1/accounts",
                                                            headers={"Authorization": 'Bearer {}'.format(
                                                                jwt.encode({'access_key': A_key,
                                                                            'nonce': str(uuid.uuid4())}, S_key))}).json())
        now_krw = float(my_exchange_account[my_exchange_account['currency'] == 'KRW']['balance'].tail())

        ## 7000원이 없으면, 5분대기 후 재실행
        if now_krw < 7000:
            time.sleep(60 * 5)
            continue

        ## 1분전 대비 10%급등한지 검증
        ohlcv_value = pyupbit.get_ohlcv(market_code, interval="minute1")
        ohlcv_value['change'] = ohlcv_value['close'] - ohlcv_value['close'].shift(1)
        ohlcv_value['change_rate'] = (ohlcv_value['close'] / ohlcv_value['close'].shift(1)) * 100

        # 지금 코인을 구매한 상태인지 체크한다.
        was_coin_bought = False

        ## 10%이상 급등했을때
        if ohlcv_value['change_rate'][-1] > 110:
            # 시장가 구매
            if round(now_krw * 0.2) >= 7000:
                ## 주문 금액을 보유 원화의 20%로 지정
                order_amount = round(now_krw * 0.2)
            else:
                order_amount = 7000

            ## API로 업비트에서 시장가 매수 진행
            buy_market_order_data = pd.DataFrame.from_dict(pyupbit.Upbit(A_key, S_key)
                                                           .buy_market_order(market_code, order_amount), orient='index').T

            # 특정 주문 조회
            ## 조회할 주문 uuid
            order_uuid = buy_market_order_data.uuid[0]
            ## API로 업비트에서 특정 주문 조회
            specific_order = pd.DataFrame.from_dict(pyupbit.Upbit(A_key, S_key).get_order(order_uuid), orient='index').T
            buy_market_order_price = (round(float(specific_order['price'][0]) / float(specific_order['executed_volume'][0]), 2))
            send_email(specific_order['market'][0] + '을 매수가 ' + str(buy_market_order_price) + '원에 ' +
                  str(round(float(specific_order['executed_volume'][0]), 5)) + '개 구매',"급등코인매수!")
            print(specific_order['market'][0] + '을 매수가 ' + str(buy_market_order_price) + '원에 ' +
                  str(round(float(specific_order['executed_volume'][0]), 5)) + '개 구매')

            was_coin_bought = True
        else:
            print(f"{market_code}은 10%이상 급등하지 않았습니다. {ohlcv_value['change_rate'][-1]}%변동")

        ## 지금 매수한 상태면 계속 매도하기 위해서 대기한다.
        while(was_coin_bought):
            time.sleep(1)

            current_price = pd.DataFrame(requests.get("https://api.upbit.com/v1/ticker?markets=" + market_code,
                                                  headers={"Accept": "application/json"}).json())

            order_quantity = pyupbit.Upbit(A_key, S_key).get_balance(market_code)   # 특정 종목 보유 수량
            ## 현재구매한 코인의 시가가 구매가보다 ascent이상이면 수익화한다.
            ascent_rate = (current_price/buy_market_order_price)*100 - 100

            if(ascent_rate >= ascent):
                sell_market_order_data = pd.DataFrame.from_dict(
                            pyupbit.Upbit(A_key, S_key).sell_market_order(market_code, order_quantity), orient='index').T
                print(str(ascent) + '% 증가하여 시장가 매도 익절')
                send_mail(str(ascent) + '% 증가하여 시장가 매도 익절', "급등주 매도 익절!")

                was_coin_bought = False

            ## 현재구매한 코인의 시가가 구매가보다 downhill미만이면 손절한다
            downhill_rate = -ascent_rate

            if(downhill_rate > downhill):
                sell_market_order_data = pd.DataFrame.from_dict(
                            pyupbit.Upbit(A_key, S_key).sell_market_order(market_code, order_quantity), orient='index').T
                print(str(downhill) + '% 하락하여 시장가 매도 손절')
                send_mail(str(downhill) + '% 하락하여 시장가 매도 손절', "급등주 매도 손절ㅠ")

                was_coin_bought = False


KRW-AAVE 109350원
KRW-AAVE은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-XLM 115원
KRW-XLM은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-HBAR 89.2원
KRW-HBAR은 10%이상 급등하지 않았습니다. 100.22471910112361%변동
KRW-ELF 245원
KRW-ELF은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-THETA 1400원
KRW-THETA은 10%이상 급등하지 않았습니다. 100.71942446043165%변동
KRW-BCH 170100원
KRW-BCH은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-ZRX 318원
KRW-ZRX은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-STPT 56.5원
KRW-STPT은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-XEM 51.8원
KRW-XEM은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-RFR 7.86원
KRW-RFR은 10%이상 급등하지 않았습니다. 100.12738853503184%변동
KRW-NU 210원
KRW-NU은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-VET 30.9원
KRW-VET은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-KAVA 1970원
KRW-KAVA은 10%이상 급등하지 않았습니다. 100.51020408163265%변동
KRW-LINK 8880원
KRW-LINK은 10%이상 급등하지 않았습니다. 100.05633802816902%변동
KRW-IOST 13.6원
KRW-IOST은 10%이상 급등하지 않았습니다. 99.27007299270073%변동
KRW-DOT 8515원
KRW-DOT은 10%이상 급등하지 않았습니다. 99.94131455399061%변동
KRW-TT 6.01원
KRW-TT은 10%이상 급등하지 않았습니다. 99.83388704318938%변동
KRW-MVL 5.52원
KRW-MVL은 10%이상 급등하지 않았습

KRW-SXP 391원
KRW-SXP은 10%이상 급등하지 않았습니다. 100.25641025641025%변동
KRW-NEAR 3055원
KRW-NEAR은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-LOOM 68.2원
KRW-LOOM은 10%이상 급등하지 않았습니다. 99.85358711566619%변동
KRW-STX 381원
KRW-STX은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-QTUM 3570원
KRW-QTUM은 10%이상 급등하지 않았습니다. 100.14025245441796%변동
KRW-TFUEL 73원
KRW-TFUEL은 10%이상 급등하지 않았습니다. 100.27472527472527%변동
KRW-SOL 29880원
KRW-SOL은 10%이상 급등하지 않았습니다. 99.96654399464704%변동
KRW-T 55.1원
KRW-T은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-MLK 394원
KRW-MLK은 10%이상 급등하지 않았습니다. 99.74683544303798%변동
KRW-IQ 8.23원
KRW-IQ은 10%이상 급등하지 않았습니다. 100.24360535931788%변동
KRW-SSX 26.1원
KRW-SSX은 10%이상 급등하지 않았습니다. 99.61832061068702%변동
KRW-ORBS 45.6원
KRW-ORBS은 10%이상 급등하지 않았습니다. 100.21978021978022%변동
KRW-ICX 289원
KRW-ICX은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-SRM 644원
KRW-SRM은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-POLYX 258원
KRW-POLYX은 10%이상 급등하지 않았습니다. 99.61389961389962%변동
KRW-BSV 55710원
KRW-BSV은 10%이상 급등하지 않았습니다. 99.92825112107623%변동
KRW-GMT 674원
KRW-GMT은 10%이상 급등하지 않았습니다. 100.14858841010401%